## Day 31 Lecture 2 Assignment

In this assignment, we will learn about the weighting and scaling with the K nearest neighbor algorithm. We will use the wine quality dataset loaded below and analyze the model generated for this dataset.

In [53]:
%matplotlib inline

import math

import matplotlib.pyplot as plt
import numpy as np
import pandas as pd

In [54]:
wine = pd.read_csv('https://raw.githubusercontent.com/Thinkful-Ed/data-science-lectures/master/wineQualityReds.csv', index_col=0)
wine.head()

,fixed.acidity,volatile.acidity,citric.acid,residual.sugar,chlorides,free.sulfur.dioxide,total.sulfur.dioxide,density,pH,sulphates,alcohol,quality
1,7.4,0.70,0.00,1.9,0.076,11.0,34.0,0.9978,3.51,0.56,9.4,5
2,7.8,0.88,0.00,2.6,0.098,25.0,67.0,0.9968,3.20,0.68,9.8,5
3,7.8,0.76,0.04,2.3,0.092,15.0,54.0,0.9970,3.26,0.65,9.8,5
4,11.2,0.28,0.56,1.9,0.075,17.0,60.0,0.9980,3.16,0.58,9.8,6
5,7.4,0.70,0.00,1.9,0.076,11.0,34.0,0.9978,3.51,0.56,9.4,5


Recall that we need to check for missing data.

In [55]:
# answer below:

wine.isnull().sum()*100/wine.isnull().count()


fixed.acidity           0.0
volatile.acidity        0.0
citric.acid             0.0
residual.sugar          0.0
chlorides               0.0
free.sulfur.dioxide     0.0
total.sulfur.dioxide    0.0
density                 0.0
pH                      0.0
sulphates               0.0
alcohol                 0.0
quality                 0.0
dtype: float64

Convert quality to a binary variable, with the dividing line between 5 and 6.

In [56]:
# answer below

wine['best'] = wine['quality'] >= 6  
wine['best'] = pd.get_dummies(wine['best'], drop_first=True)
wine.drop(columns='quality', inplace=True)

Create a train test split with 20% of the data in the test subsample.

In [57]:
# answer below:

from sklearn.model_selection import train_test_split

X = wine.drop(columns='best')
y = wine['best']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.2)
print("The number of observations in training set is {}".format(X_train.shape[0]))
print("The number of observations in test set is {}".format(X_test.shape[0]))

The number of observations in training set is 1279
The number of observations in test set is 320


Scale only the independent variables using the minmax scaler.

In [58]:
# answer below:
from sklearn.preprocessing import MinMaxScaler


scaler = MinMaxScaler()
X_train_scal = scaler.fit_transform(X_train)
X_test_scal = scaler.transform(X_test)

Create a KNN model with k=5 and report the accuracy scores. Then make a second model using the scaled data and compare your results.

In [59]:
# answer below:
from sklearn.neighbors import KNeighborsClassifier

knn = KNeighborsClassifier(n_neighbors = 5)

knn.fit(X_train, y_train)
knn.fit(X_train_scal, y_train)

print('Unscaled train score: ',knn.score(X_train, y_train))
print('Unscaled test score: ',knn.score(X_test, y_test))
print('')
print('Scaled train score: ',knn.score(X_train_scal, y_train))
print('Scaled test score: ',knn.score(X_test_scal, y_test))

Unscaled train score:  0.5175918686473807
Unscaled test score:  0.565625

Scaled train score:  0.8162627052384676
Scaled test score:  0.71875


When generating a KNN model, we can use the weighted model by setting `weights='distance'`. We can also write our own custom weights function.

Write a custom weight function that assigns the weight of 1/sqrt(distance) and use this function in your model. Report the accuracy score.

Hint: Use the `_get_weights` function in scikit learn as a resource. The code is <a href="https://github.com/scikit-learn/scikit-learn/blob/fdbaa58acbead5a254f2e6d597dc1ab3b947f4c6/sklearn/neighbors/base.py#L63" title="_get_weights">here</a>.

In [95]:
# answer below:

def weights_(dist):
        # if user attempts to classify a point that was zero distance from one
        # or more training points, those training points are weighted as 1.0
        # and the other points as 0.0
        if dist.dtype is np.dtype(object):
            for point_dist_i, point_dist in enumerate(dist):
                # check if point_dist is iterable
                # (ex: RadiusNeighborClassifier.predict may set an element of
                # dist to 1e-6 to represent an 'outlier')
                if hasattr(point_dist, '__contains__') and 0. in point_dist:
                    dist[point_dist_i] = point_dist == 0.
                else:
                    dist[point_dist_i] = 1. / point_dist
        else:
            with np.errstate(divide='ignore'):
                dist = 1. / np.sqrt(dist)
            inf_mask = np.isinf(dist)
            inf_row = np.any(inf_mask, axis=1)
            dist[inf_row] = inf_mask[inf_row]
        return dist


In [93]:
knn1 = KNeighborsClassifier(n_neighbors = 5, weights=weights_)

knn1.fit(X_train_scal, y_train)
knn1.score(X_train_scal, y_train)

1.0

In [94]:
knn1.score(X_test_scal, y_test)

0.775